<a href="https://colab.research.google.com/github/rpenal/Algoritmos2022-I/blob/main/Lab4/Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmo de fuerza bruta con precisión $x=raíz\pm 0.0001$

In [ ]:
#Fuerza bruta COMPLEJIDAD O(n) tiempo de ejecuci+on 48s
#La complejidad de la función signo(x) es 0(1)
import time
def signo(x):
  if x<=0:  #Evaluar si un número es menor a 0 es O(1)
    return -1
  elif x>0: #Evaluar si un número es mayor a 0 es O(1)
    return 1

#La complejidad de la función búsqueda es O(n)
def busqueda(polinom,a,b,distancia):
  #x_a_explorar es una función anónima que nos permite definir una sucesión de números entre a y b con distancia=distancia
  x_a_explorar = lambda i: a+i*distancia
  i=0
  #n es el número de elementos de la sucesión
  n = int((b-a)/distancia)
  #Iteramos sobre el total de números de la secuencia, por ello hacemos un while hata i menos que n
  #El costo del bloque dentro del bucle es O(n)
  while i<n:
    #Como sabemos si f(x) y f(y) tienen signo contrario y x<y, entonces existe una raíz entre x y y.
    #Como solo una pareja contigua de la sucesión tiene esta propiedad, (x+y)/2, aproximará el valor de la raíz
    #Evaluar si el signo de la multiplicación de la pareja es negativo tiene costo O(1)
    if signo(polinom(x_a_explorar(i))*polinom(x_a_explorar(i+1)))==-1:
      #Nos permite obtener la raíz con una precisión de 0.0001 en x
        print(f'Raíz en {(x_a_explorar(i)+x_a_explorar(i+1))/2}')
    i+=1

#Definimos nuestro polinomio como una función anónima lambda
#Evaluar polinom(x) es O(1)
polinom = lambda x: x**5-59*x**4+35*x**3-250*x**2+x-70
tiempo_inicial = time.time()
#Encuentra las raíces del polinomio x: x**5-59*x**4+35*x**3-250*x**2+x-70 entre -1000 y 1000 con una distancia en x de máx 0.0001, O(n)
busqueda(polinom,-1000,1000,distancia=0.0001)
tiempo_final = time.time()
print(f"Se ha ejecutado en {tiempo_final-tiempo_inicial} s")

Raíz en 58.47455000000002
Se ha ejecutado en 42.50424790382385 s


# Algoritmo de bisección

In [ ]:
# Recursivo bisección
# Programa que ejecuta el algoritmo voraz de forma recursiva
import time

#Función que inicializa el programa y genera el ciclo del intervalo de -1000 a 1000
def main():
    # Dominio
    a = -1000
    b = 1000

    # Iteración por números enteros
    for i in range(a, b):
        # Comprueva un salto de positivo a negativo o de negativo a positivo
        if ((f(i) > 0) and (f(i+1) < 0)) or ((f(i) < 0) and (f(i+1) > 0)):
            print(eval(i, i+1))

        if f(i) == 0:
          print(i)
    
#Función que evlúa la función suministrada
def f(x):
    #x^5-59x^4+35x^3-250x^2+x-70=0
    y =  ((x ** 5) - (59 * (x ** 4)) + (35 * (x ** 3)) - (250 * (x ** 2)) + x - 70)     # Ecuación a evaluar por la guía
    #y = ((321 * (x ** 3)) + (654 * (x ** 2)) - (25 * x) - 63)                          # Ecuación con mas de una raiz
    return y


#Función que evalua si se está en en la raíz de la ecuación
def eval(a, b):
    #Punto medio
    c = a + abs(b - a)/2.0

    # Se evalua f con respecto a el minimo, el maximo y el punto medio del dominio respectivamente "f(x)"
    u = f(a)
    v = f(b)
    w = f(c)
         
    # Comprueva un salto de positivo a negativo o de negativo a positivo con respecto a f(x)
    if (u < 0 and w > 0) or (u > 0 and w < 0):
        # Función recursiva
        return eval(float('{0:.12f}'.format(a)), float('{0:.12f}'.format(c)))
    
    # Comprueva un salto de positivo a negativo o de negativo a positivo
    if (w < 0 and v > 0) or (w > 0 and v < 0):
        # Función recursiva
        return eval(float('{0:.12f}'.format(c)), float('{0:.12f}'.format(b)))
    
    # Si los 10 decimales de a y b son iguales retorna a 
    if float('{0:.10f}'.format(a)) == float('{0:.10f}'.format(b)):
        return a

#Ejecución del programa
if __name__ == "__main__":
    start_time = time.time()
    main()
    end_time = (time.time() - start_time)
    print("----- " + str(end_time) + " segundos -----")
    # Complejidad O(n*log(n))

58.474565066319
----- 0.021967172622680664 segundos -----


# Curiosidades: Algoritmo de Fuerza Bruta con mayor precisión sobre el valor de y

In [ ]:
%%writefile clrootsbf.c
#include <CL/cl.h>
#include <stdio.h>

#define STEPS 1000000000000
#define WORK_GROUPS 1000
#define START_RANGE 58
#define END_RANGE 59

const char *KERNEL = "double formula(double x) {"
                     "  return pow(x, 5) - 59 * pow(x, 4) + 35 * pow(x, 3) - "
                     "250 * pow(x, 2) + x - 70;"
                     "}"
                     ""
                     "kernel void roots(const double start, const double step) {"
                     "  size_t id = get_global_id(0);"
                     "  double x = start + ((step) * id);"
                     "  double result = formula(x);"
                     "  if (result > -0.0001 && result < 0.0001) {"
                     "    printf(\"Root found at x = %.17g\\n\", x);"
                     "  }"
                     "}";

void findRoots(double (*formula)(double), double x1, double x2,
               unsigned long num_op) {
  if (x1 > x2) {
    return;
  }

  double step = num_op ? ((-x1 + x2) * 2) / num_op : 1;

  for (double i = x1; i <= x2; i += step) {
    double result = formula(i);
    // printf("%f\n", result);
    if (result > -0.0001 && result < 0.0001) {
      printf("%.17g\n", i);
    }
  }
}

int main() {
  int cl_err = 0;

  double start = START_RANGE;
  double end = END_RANGE;
  double step = (END_RANGE - START_RANGE) / (double) STEPS;
  printf("Step for %ld operations in range [%f, %f] is %.17f\n", STEPS, start, end, step);

  size_t global = STEPS;
  size_t local = WORK_GROUPS;
  size_t offset = 2;

  cl_platform_id platform_id;
  cl_device_id device_id;
  cl_context context;
  cl_command_queue commands;
  cl_program program;
  cl_kernel kernel;

  cl_err = clGetPlatformIDs(1, &platform_id, NULL);
  if (cl_err != CL_SUCCESS) {
    printf("Error: Failed to get platform id!\n");
    return EXIT_FAILURE;
  }

  cl_err = clGetDeviceIDs(platform_id, CL_DEVICE_TYPE_ALL, 1, &device_id, NULL);
  if (cl_err != CL_SUCCESS) {
    printf("Error: Failed to create a device group!\n");
    return EXIT_FAILURE;
  }

  context = clCreateContext(NULL, 1, &device_id, NULL, NULL, &cl_err);
  if (cl_err != CL_SUCCESS) {
    printf("Error: Failed to create a compute context!\n");
    return EXIT_FAILURE;
  }

  commands =
      clCreateCommandQueueWithProperties(context, device_id, NULL, &cl_err);
  if (cl_err != CL_SUCCESS) {
    printf("Error: Failed to create a command queue!\n");
    return EXIT_FAILURE;
  }

  program = clCreateProgramWithSource(context, 1, &KERNEL, NULL, &cl_err);
  if (cl_err != CL_SUCCESS) {
    printf("Error: Failed to create compute program!\n");
    return EXIT_FAILURE;
  }

  cl_err = clBuildProgram(program, 0, NULL, NULL, NULL, NULL);
  if (cl_err != CL_SUCCESS) {
    size_t len;
    char *log;

    printf("Error: Failed to build program executable!\n");
    clGetProgramBuildInfo(program, device_id, CL_PROGRAM_BUILD_LOG, 0, NULL,
                          &len);

    log = malloc(len);

    clGetProgramBuildInfo(program, device_id, CL_PROGRAM_BUILD_LOG, len, log,
                          &len);

    printf("%s\n", log);
    free(log);
    return EXIT_FAILURE;
  }

  kernel = clCreateKernel(program, "roots", &cl_err);
  if (cl_err != CL_SUCCESS) {
    printf("Error: Failed to create compute kernel!\n");
    return EXIT_FAILURE;
  }

  cl_err = clSetKernelArg(kernel, 0, sizeof(double), &start);
  cl_err |= clSetKernelArg(kernel, 1, sizeof(double), &step);
  if (cl_err != CL_SUCCESS) {
    printf("Error: Failed to set kernel arguments!\n");
    return EXIT_FAILURE;
  }

  cl_err = clEnqueueNDRangeKernel(commands, kernel, 1, &offset, &global, &local,
                                  0, NULL, NULL);
  if (cl_err != CL_SUCCESS) {
    printf("Error: Failed to execute kernel!\n");
    return EXIT_FAILURE;
  }

  clFinish(commands);
  clReleaseKernel(kernel);
  clReleaseProgram(program);
  clReleaseCommandQueue(commands);
  clReleaseContext(context);
  clReleaseDevice(device_id);

  return 0;
}

In [ ]:
!gcc -Wall -Wextra clrootsbf.c -o clrootsbf -lOpenCL -D CL_TARGET_OPENCL_VERSION=300 -O3
!time ./clrootsbf